In [14]:
#Import Necessary Modules
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json

In [15]:
source = requests.get('https://www.btk.gov.tr/yillik-il-istatistikleri')

In [16]:
soup = BeautifulSoup(source.text, 'lxml')

In [17]:
files = soup.find_all(name='div', attrs={'class':'SmTqk'})[0]

In [18]:
links = {}
for a in files.find_all(name='a'):
    link = a['href']
    if link.endswith('.xlsx'):
        links[a.text] = link

In [19]:
links

{'2016 - 2021': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni-v1.xlsx',
 '2015 - 2020': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/elektronik-haberlesme-sektorune-iliskin-il-bazinda-yillik-istatistik-bulteni.xlsx',
 '2014 - 2019': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni-20-05-2020.xlsx',
 '2013 - 2018': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni1.xlsx',
 '2012 - 2017': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni.xlsx',
 '2011 - 2016': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2011-2016-5a3a48147c468.xlsx',
 '2010 - 2015': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2010-2015.xlsx',
 '2009 - 2014': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2009-2014.xlsx',
 '2008 - 2013': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2

In [20]:
def get_dataframe(link):
    data = pd.read_excel(link, sheet_name='İl Bazlı Veri')
    data = data.iloc[2:,:]
    data['detail'] = data.iloc[:,1].fillna('') + data.iloc[:,2].fillna('')
    data['city'] = data.iloc[:,0]
    data = data.iloc[:,3:]
    data['city'] = data['city'].fillna(method='ffill')
    data.dropna(subset=[x for x in data.columns if str(x) not in ['city','detail']], axis=0, inplace=True)
    data['detail'] = data['detail'].str.strip(r'[()]')
    return data[data.columns[::-1]].reset_index(drop=True)[:-1]

In [61]:
df = pd.DataFrame()
for key, link in links.items():
    print(key)
    data = get_dataframe(link)
    if df.shape[0] == 0:
        df = data
    else:
        cols_to_use = data.columns.difference(df.columns[2:])
        df = df.merge(data[cols_to_use], how='outer', on=['city', 'detail'])

2016 - 2021
2015 - 2020
2014 - 2019
2013 - 2018
2012 - 2017
2011 - 2016
2010 - 2015
2009 - 2014
2008 - 2013
2007 - 2012


In [62]:
df

,city,detail,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,TOPLAM,Population,84680273.0,83614362.0,83154997.0,82003882.0,80810525.0,79814871,78741053,77695904,76667864,75627384,74724269,73722988,72561312,71517100,70586256
1,TOPLAM,Number of Fixed Telephony Access Lines,12310016.0,12448604.0,11532903.0,11633461.0,11308444.0,11077559,11493056.7552,12528865,13551705,13859672,15210846,16201466,16534356,17502205,18201006
2,TOPLAM,Santral Capacity for Fixed Telephony,16909937.0,15397373.0,16228650.0,18441347.0,20545118.0,21408987,23273061,24492204,25038005,23209666,23425805,22637734,22074418,22921096,21772391
3,TOPLAM,Number of Payhones,45445.0,49861.0,55056.0,63693.0,66894.0,75714,81249,83209,87227,89970,88527,87452,89479,87559,87217
4,TOPLAM,Number of Mobile Telephony Subscriptions - Total,86288834.0,82128104.0,80790877.0,80117999.0,77903347.0,75061699,73639261,71888416,69661108,67680547,65321745,61769635,62779556,65287049,61975807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,YALOVA,Number of Mobile Telephony Subscriptions - 3G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71192,NaN,NaN,NaN
1438,KARABÜK,Number of Mobile Telephony Subscriptions - 3G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51616,NaN,NaN,NaN
1439,KİLİS,Number of Mobile Telephony Subscriptions - 3G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29497,NaN,NaN,NaN
1440,OSMANİYE,Number of Mobile Telephony Subscriptions - 3G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147056,NaN,NaN,NaN


In [63]:
df.sort_values(by='city', inplace=True)
df.reset_index(drop=True, inplace=True)

In [64]:
df = df.melt(id_vars=['city','detail'], var_name=['year'], value_name='value')

In [65]:
df = df.pivot(index=['city','year'], columns='detail').reset_index()

In [66]:
df.columns = df.columns[:2].get_level_values(0).tolist() + df.columns[2:].get_level_values(1).tolist()

In [67]:
df.columns = [col.lower() for col in df.columns]

In [68]:
df.head()

,city,year,cable,fiber,mobile broadband dedicated,number of broadband subscriptions - total,number of cable tv subscriptions,number of fixed broadband subscriptions - total,number of fixed telephony access lines,number of mobile broadband subscriptions - total,...,number of mobile telephony subscriptions - 2g,number of mobile telephony subscriptions - 3g,number of mobile telephony subscriptions - total,number of payhones,other,population,santral capacity for fixed telephony,standard mobile broadband,the length of fiber-km,xdsl
0,ADANA,2007,NaN,NaN,NaN,NaN,41115,NaN,402348,NaN,...,NaN,NaN,1759062,2022,NaN,2006650,523425,NaN,NaN,NaN
1,ADANA,2008,NaN,NaN,NaN,NaN,40400,NaN,369043,NaN,...,NaN,NaN,1889383,1986,NaN,2026319,534366,NaN,NaN,NaN
2,ADANA,2009,NaN,NaN,NaN,NaN,43826,NaN,332356,NaN,...,NaN,NaN,1768410,1912,NaN,2062226,525061,NaN,NaN,NaN
3,ADANA,2010,NaN,NaN,NaN,NaN,51835,NaN,310454,NaN,...,1172924,542370,1715294,1877,NaN,2085225,529486,NaN,NaN,NaN
4,ADANA,2011,30406,21286,29052,561382.40352,57089,167692.40352,270622,393690,...,983747,NaN,1830386,1786,126.40352,2108805,596315,364638,NaN,115874


In [81]:
numeric_cols = df.select_dtypes(include=np.number).columns
df[numeric_cols] = df.fillna(value=0).loc[:, numeric_cols].astype('int')

In [83]:
df.to_csv('final_merged.csv', encoding='utf-8-sig', index=False)

In [84]:
df = pd.read_csv('final_merged.csv')

In [85]:
df.head()

,city,year,cable,fiber,mobile broadband dedicated,number of broadband subscriptions - total,number of cable tv subscriptions,number of fixed broadband subscriptions - total,number of fixed telephony access lines,number of mobile broadband subscriptions - total,...,number of mobile telephony subscriptions - 2g,number of mobile telephony subscriptions - 3g,number of mobile telephony subscriptions - total,number of payhones,other,population,santral capacity for fixed telephony,standard mobile broadband,the length of fiber-km,xdsl
0,ADANA,2007,0,0,0,0,41115,0,402348,0,...,0,0,1759062,2022,0,2006650,523425,0,0,0
1,ADANA,2008,0,0,0,0,40400,0,369043,0,...,0,0,1889383,1986,0,2026319,534366,0,0,0
2,ADANA,2009,0,0,0,0,43826,0,332356,0,...,0,0,1768410,1912,0,2062226,525061,0,0,0
3,ADANA,2010,0,0,0,0,51835,0,310454,0,...,1172924,542370,1715294,1877,0,2085225,529486,0,0,0
4,ADANA,2011,30406,21286,29052,561382,57089,167692,270622,393690,...,983747,0,1830386,1786,126,2108805,596315,364638,0,115874


In [86]:
df[df['city']=='ADANA']

,city,year,cable,fiber,mobile broadband dedicated,number of broadband subscriptions - total,number of cable tv subscriptions,number of fixed broadband subscriptions - total,number of fixed telephony access lines,number of mobile broadband subscriptions - total,...,number of mobile telephony subscriptions - 2g,number of mobile telephony subscriptions - 3g,number of mobile telephony subscriptions - total,number of payhones,other,population,santral capacity for fixed telephony,standard mobile broadband,the length of fiber-km,xdsl
0,ADANA,2007,0,0,0,0,41115,0,402348,0,...,0,0,1759062,2022,0,2006650,523425,0,0,0
1,ADANA,2008,0,0,0,0,40400,0,369043,0,...,0,0,1889383,1986,0,2026319,534366,0,0,0
2,ADANA,2009,0,0,0,0,43826,0,332356,0,...,0,0,1768410,1912,0,2062226,525061,0,0,0
3,ADANA,2010,0,0,0,0,51835,0,310454,0,...,1172924,542370,1715294,1877,0,2085225,529486,0,0,0
4,ADANA,2011,30406,21286,29052,561382,57089,167692,270622,393690,...,983747,0,1830386,1786,126,2108805,596315,364638,0,115874
5,ADANA,2012,29739,33984,33939,688428,54652,184278,223992,504150,...,755860,0,1887012,1705,149,2125635,642813,470211,5205,120406
6,ADANA,2013,25015,44191,34120,818237,48198,196773,244392,621464,...,604306,0,1950911,1516,208,2149260,653019,587344,5760,127359
7,ADANA,2014,27355,51347,26638,1083065,47642,210073,217236,872992,...,365313,0,2002073,1379,219,2165595,470088,846354,6339,131152
8,ADANA,2015,30139,58766,29738,1280605,46546,225798,191438,1054807,...,421704,0,2032485,1275,221,2183167,452914,1025069,6887,136672
9,ADANA,2016,35442,66005,24895,1703083,48102,251442,185835,1451641,...,109352,0,2075830,1187,653,2201670,335232,1426746,6929,149342
